In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

In [2]:
yf.pdr_override()

In [3]:
df = pd.read_csv("stocksymbols - Sheet1.csv")
syms = list(df['Symbol'])
print(syms)

['SBIN.NS', 'AXISBANK.BO', 'IOC.BO', 'NMDC.BO', 'KRITIIND.BO', 'TATAPOWER.NS', 'AUROLAB.BO', 'RESONANCE.BO', 'ANDHRAPET.BO', 'MANALIPETC.BO', 'GAIL.NS', 'SAIL.NS', 'HDFCBANK.BO', 'ONGC.NS', 'INFY.BO', 'BHAGYAPROP.BO', 'FEDERALBNK.NS', 'WIPRO.BO', 'IDEA.NS', 'ALOKTEXT.BO', 'BANKBARODA.BO', 'BHEL.NS', 'DISHTV.BO', 'VEDL.BO', 'PTC.NS', 'PRESSMN.NS', 'VEDL.BO']


In [4]:
start = "2016-04-01"
end = str(int(start[:4]) + 3) + start[4:]
stock_data = pdr.get_data_yahoo(syms[:5], start = start, end = end)['Adj Close']
stock_data.head()

[*********************100%***********************]  5 of 5 completed


,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
Date,,,,,
2016-04-01,440.503143,70.593666,32.958870,79.307602,191.382736
2016-04-04,436.142731,72.174149,33.401604,79.712654,190.453461
2016-04-05,423.551331,72.101608,33.352413,79.186081,180.329224
2016-04-06,416.300262,73.744942,32.466949,82.264420,179.840118
2016-04-07,417.868042,73.790176,31.729065,82.588448,177.981552


In [5]:
returns = stock_data.pct_change()
returns.head()

,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
Date,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN
2016-04-04,-0.009899,0.022388,0.013433,0.005107,-0.004856
2016-04-05,-0.028870,-0.001005,-0.001473,-0.006606,-0.053159
2016-04-06,-0.017120,0.022792,-0.026549,0.038875,-0.002712
2016-04-07,0.003766,0.000613,-0.022727,0.003939,-0.010335


In [6]:
mean_daily_returns = np.array(returns.mean()).reshape(-1, 1)
cov = returns.cov()

In [7]:
mean_daily_returns

array([[0.00092921],
       [0.00191748],
       [0.00041526],
       [0.00045104],
       [0.00090449]])

In [8]:
cov

,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
AXISBANK.BO,0.000313,0.000051,0.000045,0.000082,0.000153
IOC.BO,0.000051,0.002062,-0.000006,0.000056,0.000058
KRITIIND.BO,0.000045,-0.000006,0.001180,0.000117,0.000066
NMDC.BO,0.000082,0.000056,0.000117,0.000372,0.000123
SBIN.NS,0.000153,0.000058,0.000066,0.000123,0.000442


In [9]:
srs = []
portfolio_stds = []
rand_wts = []
portfolio_returns = []
risk_free_rate = 0
for i in range(0, 20000):
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    rand_wts.append(random_weights)
    # portolfio return
    portfolio_return = np.sum(mean_daily_returns * random_weights)*252
    portfolio_returns.append(portfolio_return)
    # portfolio volatility
    portfolio_std = np.sqrt(np.dot(random_weights.T, np.dot(cov, random_weights))) * np.sqrt(252)
    portfolio_stds.append(portfolio_std)
    # sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    srs.append(sharpe_ratio)
max_index = srs.index(max(srs))
best_wts = rand_wts[max_index]
max_sr = srs[max_index]
portfolio_sd = portfolio_stds[max_index]
max_return = portfolio_returns[max_index]
print(f"Max Sharpe Ratio: {max_sr}")
print(f"Best Weights: {best_wts.T}")
print(f"Portfolio Volatility: {portfolio_sd*100}%, Sum of weights: {np.sum(best_wts)}")
print(f"Max Expected Annual Return: {max_return*100}%")

Max Sharpe Ratio: [[1.10061387]]
Best Weights: [[0.47368903 0.20116718 0.03190962 0.05879166 0.2344425 ]]
Portfolio Volatility: [[24.67554403]]%, Sum of weights: 1.0
Max Expected Annual Return: 27.158246076400733%
